In [ ]:
import ipyvuetify as v
import ipywidgets as widgets
from utils import utils
from scripts import run_sepal_process
from scripts import results
import matplotlib.pyplot as plt


In [ ]:
#for testing 
#year = '2019'
#asset_name = "users/bornToBeAlive/Glad_Singapore"

In [ ]:
sepal_process_output = v.Alert(children=[''], type='info', text=True, class_="mt-5")
#widget_alert
#fig = plt.figure('results')
sepal_process_result = v.Flex(xs12=True, class_='pa-0', children=['no result yet'])

In [ ]:
#Messages
NO_INPUT = "No input have been selected, provide them through step 1."

In [ ]:
#validate the selected data

sepal_process_btn = v.Btn(
    color='primary', 
    children=[
        v.Icon(left=True, children=['mdi-cogs']),
        'Run SEPAL process'
    ]
)
sepal_validate_data = v.Container(children=[sepal_process_btn])

def sepal_on_click(widget, event, data):
    
    global asset_name
    global year
    global sepal_process_output
    global sepal_process_result
    
    if asset_name == None or year == None:
        utils.displayIO(sepal_process_output, 'error', NO_INPUT)
        return 
    
    #do something 
    utils.toggleLoading(sepal_process_btn)
    run_sepal_process.run_sepal_process(asset_name, year, sepal_process_output)
    utils.toggleLoading(sepal_process_btn) 
    
    #display the map
    figure = results.display_graph(asset_name, year)
    sepal_process_result.children = [figure]

sepal_validate_data.on_event('click', sepal_on_click)
#widget_validate_data

In [ ]:
#create the display 
inputs = v.Layout(
    _metadata={'mount-id': 'data-input'},
    row=True,
    class_="pa-5",
    align_center=True, 
    children=[
        v.Flex(xs12=True, children=[sepal_validate_data]),
        v.Flex(xs12=True, children=[sepal_process_output])
    ]
)
#inputs

SEPAL_content_main =  v.Layout(
    _metadata={'mount_id': 'widget_sepal_process'},
    Row=True,
    xs12=True,
    align_center=True, 
    class_="ma-5",
    children=[
        v.Card( 
            class_="pa-5",
            raised=True,
            xs12=True,
            children=[
                v.Html(xs12=True, tag='h2', children=['Run the SEPAL process']),
                v.Flex(xs12=True, children=[inputs]),   
            ]
        ) 
    ]
)

SEPAL_results = v.Layout(
    _metadata={'mount_id': 'widget_result'},
    Row=True,
    xs12=True,
    align_center=True,
    class_="ma-5",
    children=[
        v.Card( 
            class_="pa-5",
            raised=True,
            xs12=True,
            children=[
                v.Html(xs12=True, tag='h2', children=['data Result']),
                sepal_process_result  
            ]
        )
    ]
)
SEPAL_content_main
SEPAL_results